In [1]:
# Package Import
import pandas as pd
import numpy as np
import re
import json 

# Path
path_raw = r'C:/Users/Usuario/Documents/b4d/data/raw/'
path_processed = r'C:/Users/Usuario/Documents/b4d/data/processed/'

# Name of file
file_name = 'fairplay_connectly_report_wk2'

# file_name_base = 'tigobo_1804-Internet Inicial 2-Wpp'

# File extension
file_extension = '.csv'

# Suffix to proceseed files
suffix = '_clean'

In [2]:
# Reading reporte csv
# Columns
columns = ['customer_external_id', 'campaign_name', 'dispatched_at', 'sent_at', 'delivered_at', 'read_at', 'error_code', 'error_msg', 'link_clicks']

# Reading csv to dataframe
df = pd.read_csv(path_raw+file_name+file_extension, usecols=columns)
print(f"Shape:  {df.shape}")

Shape:  (1196, 9)


In [ ]:
# (Optional)
# Reading base/original csv
# Columns
columns_base = ['PhoneNumber']

# Reading csv to dataframe
df_base = pd.read_csv(path_raw+file_name_base+file_extension, usecols=columns_base)
print(f"Shape:  {df_base.shape}")


In [ ]:
# (Optional)
# Analysis 1.1
# why some phone number was not sent the difussion message
df_diff = df_base[~df_base['PhoneNumber'].isin(df['customer_external_id'])]
print(f"Shape:  {df_diff.shape}")

suffix_diff = '_not_merged'

In [ ]:
# Fix 1.1 - Febrero
# replace name test campaing in order to follow the rule
#df['campaign_name'] = np.where(df['campaign_name']=='Cambio de caracteristicas de planes movil Postpago_Mega Plan 169_21022025', 'Cambio De Caracteristicas De Planes Movil Postpago Mega Plan 169 21022025 . Prueba',df['campaign_name'])
#df.shape

In [ ]:
# Fix 2.1
# Exclude some campaigns beacuse it was error sending
#df = df[df['campaign_name']!='Comunicacion Por Whatsapp Recordatorio Cambio De Caracteristicas Movil Mega Plan 180 Mega Plan 225 Mega Plan 299 Mega Plan 305 . Planes Movil Postpago Mega Plan 299 . 21 02 25 . Lote']
#df.shape

In [ ]:
#
df = df[df['campaign_name'].isin(['Datos Empresa Oc Mensajeria . Bridgeconnectly web . 23 05 25 . Prueba', 'Datos Empresa Oc Mensajeria . Bridgeconnectly Wapp . 23 05 25 . Lote2', 'Datos Empresa Oc Mensajeria . Bridgeconnectly wapp . 23 05 25 . lote1'])]

In [3]:
# Upper name for all name campaigns
df['campaign_name'] = df['campaign_name'].str.upper()

In [4]:
# Formating the campaign name
# Regexs
#regex_1 = r'\s*?\.?\s*?(BATCH|NORMAL|CX|LOTE|LOT|LO|COPY).*$'
regex_1 = r'\s*?\.?\s*?(BATCH|NORMAL|CX|LOTE|LOT|COPY).*$'
regex_2 = r'\.\s\d{2}$'
regex_3 = r'\s*?\.?\s*?(OK|ENVIO OK|ENVIO|PRUEBA INTERNA|PRUEBA)\s?\d*?$'
regex_4 = r'\.?\s?PRUEBA\s?'

# Function to extraction string with regex
def f_regex(data_str, regex, str_replace=''):
    return re.sub(regex, str_replace, data_str, flags=re.IGNORECASE)

# Applying the regex with the campaing name column
df['campaign_name_base'] = df['campaign_name'].apply(lambda x: f_regex(x, regex_1))
df['campaign_name_base'] = df['campaign_name_base'].apply(lambda x: f_regex(x, regex_2))
df['campaign_name_base'] = df['campaign_name_base'].apply(lambda x: f_regex(x, regex_3))
df['campaign_name_base'] = df['campaign_name_base'].apply(lambda x: f_regex(x, regex_4))

# Verify if this campaign is test
# df['tipo'] = np.where(df['campaign_name'].str.contains('PRUEBA', na=False), 'PRUEBA', 'LOTE')

# Filter to verify if this message was error
# This filter was used in excel and below is equivalent with pandas
# =O(ESBLANCO([@[error_code]]);NO(ESBLANCO([@[read_at]]));NO(ESBLANCO([@[delivered_at]]));NO(ESBLANCO([@[sent_at]])))
filter_delivered = (
   # df['error_code'].isna() |                  
    df['read_at'].notna() |                    
    df['delivered_at'].notna() 
   # | df['sent_at'].notna()                      
)
df['delivered'] = np.where(filter_delivered, 'TRUE', 'FALSE')

df['read'] = np.where(df['read_at'].notna(), 'TRUE', 'FALSE')

# old code
#df['estado'] = ~((df['error_code'] == '') | ~(df['error_code'].isna()))
# df.dtypes

In [ ]:
# regex_5 = r'Compra por Whatsapp'

# df['buy_wsp'] = np.where(df['link_clicks'].str.contains(regex_5, case=True, regex=True, na=False), 'TRUE', 'FALSE')

# regex_6 = r'Compra por web'

# df['buy_web'] = np.where(df['link_clicks'].str.contains(regex_6, case=True, regex=True, na=False), 'TRUE', 'FALSE')

In [5]:
df['link_clicks'] = df['link_clicks'].astype(str)

In [6]:
def get_first_name(json_str):
    try:
        data = json.loads(json_str)
        data_sorted = sorted(data, key=lambda x: x["ts"])
        return data_sorted[0]["name"] if isinstance(data_sorted, list) and len(data_sorted) > 0 else None
    except (json.JSONDecodeError, KeyError):
        return None
    
df['buy_wsp'] = np.where(df['link_clicks'].apply(get_first_name) == "Compra por Whatsapp", "TRUE", "FALSE")
df['buy_web'] = np.where(df['link_clicks'].apply(get_first_name) == "Compra por web", "TRUE", "FALSE")

In [7]:

df["buy_wsp_count"] = df["link_clicks"].apply(
    lambda x: sum(1 for event in json.loads(x) if isinstance(event, dict) and event["name"] == "Compra por Whatsapp")
    if x != 'nan' and pd.notna(x) and x.strip() != "" else 0
)

df["buy_web_count"] = df["link_clicks"].apply(
    lambda x: sum(1 for event in json.loads(x) if isinstance(event, dict) and event["name"] == "Compra por web")
    if x != 'nan' and pd.notna(x) and x.strip() != "" else 0
)

In [8]:
# 
df['engagement'] = df["buy_wsp_count"] + df["buy_web_count"]

In [9]:
# 
def date_assign(name):
    if name in ['DATOS EMPRESA OC MENSAJERIA . BRIDGECONNECTLY WAPP . 23 05 25 . LOTE3']:
        return pd.to_datetime('2025-05-26 12:00')
    elif name in ['DATOS EMPRESA OC MENSAJERIA . BRIDGECONNECTLY WEB . 23 05 25 . LOTE2']:
        return pd.to_datetime('2025-05-26 12:00')
    else:
        return pd.NaT 
    
df['date'] = df['campaign_name'].apply(date_assign)

In [10]:
df['day'] = df['date'].dt.day_name(locale='es_ES')
df['hour'] = df['date'].dt.strftime('%H:%M:%S')

In [11]:
# Count message for each campaigns
#df.groupby('campaign_name_base', as_index=False).size()
df_tmp = df.copy()
df_tmp['dummy'] = 1
result = pd.crosstab(
        [df_tmp['campaign_name_base']], 
        df_tmp['tipo'], 
        margins=True
    ).reset_index()
result


KeyError: 'tipo'

In [ ]:
# Total by error code
#df.groupby(['error_code', 'error_msg'], as_index=False).size()

df_tmp = df.copy()
df_tmp['dummy'] = 1
result = pd.crosstab(
        [df_tmp['error_code'], df_tmp['error_msg']], 
        df_tmp['dummy'], 
        margins=True
    ).reset_index()
result


In [ ]:
# Total by estado
#df.groupby('estado', as_index=False).size()

df_tmp = df.copy()
df_tmp['dummy'] = 1
result = pd.crosstab([df_tmp['estado']],df_tmp['dummy'],margins=True)
result

In [ ]:
# Change error code
# Its not important
# df['error_code'] = np.where(df['error_code']=='ERROR_CODE_NOT_ELIGIBLE_FOR_DELIVERY',131049,df['error_code'])
# df.shape

In [12]:
# Format type of data column
df['sent_at'] = pd.to_datetime(df['sent_at'], errors='coerce')
df['dispatched_at'] = pd.to_datetime(df['dispatched_at'], errors='coerce')
df['delivered_at'] = pd.to_datetime(df['delivered_at'], errors='coerce')
df['read_at'] = pd.to_datetime(df['read_at'], errors='coerce')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['error_code'] = np.floor(pd.to_numeric(df['error_code'], errors='coerce')).astype('Int64') #df['error_code'].astype('Int64')

#df['Tipo'] = np.where(df['tipo'], "PRUEBA", "LOTE")
#df['Estado'] = np.where(df['estado'], "VERDADERO", "FALSO")

In [13]:
# drop column
df.drop('campaign_name', axis=1, inplace=True)

# reorder columns 
df = df[['customer_external_id', 'campaign_name_base', 'dispatched_at', 'sent_at',
       'delivered_at', 'read_at', 'error_code', 'error_msg', 'link_clicks', 'delivered',
       'read', 'engagement', 'buy_wsp', 'buy_web', 'buy_web_count', 'buy_wsp_count', 'date',
       'day', 'hour']]
df.shape

(1196, 19)

In [ ]:
df.tail(2)

In [14]:
# format datetime
df['sent_at'] = df['sent_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['dispatched_at'] = df['dispatched_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['delivered_at'] = df['delivered_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['read_at'] = df['read_at'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['date'] = df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [15]:
# Export as csv
# df.to_csv(path_processed+file_name+suffix+file_extension, encoding='utf-8', index=False)

df.to_excel(path_processed+file_name+suffix+'.xlsx', index=False)


In [ ]:
# (Opcional)
# Export as csv
df_diff.to_csv(path_processed+file_name_base+suffix_diff+file_extension)